# Convergence

In [ ]:
import matplotlib.pyplot as plt
root_folder = '/data/LiftFly3D/DF3D/'

def read_convergence_info(file):
    f=open(file, "r")
    contents=f.readlines()
    epoch, lr, loss_train, loss_test, err_test = [], [], [], [], []
    for i in range(1,len(contents)):
        line = contents[i][:-1].split('\t')
        epoch.append(float(line[0]))
        lr.append(float(line[1]))
        loss_train.append(float(line[2]))
        loss_test.append(float(line[3]))
        err_test.append(float(line[4]))
        
    return epoch, lr, loss_train, loss_test, err_test

for cam in ['cam_0', 'cam_1', 'cam_2']:
    epoch, lr, loss_train, loss_test, err_test = read_convergence_info(root_folder + cam +'/log_train.txt')
    plt.plot(err_test)
    
#plt.xscale('log')
plt.xlim([1, 100]) 

# Errors

In [ ]:
root_folder = '/home/gosztolai/Dropbox/github/fly_data_analysis/LiftFly3D/DF3D'

def read_convergence_info(file):
    f=open(file, "r")
    contents=f.readlines()
    errH, errM, errF, errtotal = [], [], [], []
    for i in range(1,len(contents)):
        line = contents[i].split('\t')
        errH.append(float(line[0]))
        errM.append(float(line[1]))
        errF.append(float(line[2]))
        if i < 4:
            errtotal.append(float(line[3]))
        
    return errH, errM, errF, errtotal

errH, errM, errF, errtotal = read_convergence_info(root_folder + '/cam_angle_test_errors.txt')        

In [ ]:
import numpy as np
np.array(errtotal)

# Videos

In [7]:
import cv2
import numpy as np
import glob
import pickle
import matplotlib.pyplot as plt
from skeleton import skeleton
from matplotlib.animation import FFMpegWriter
import matplotlib
matplotlib.use("Agg")

Folders

In [2]:
experiment = '180920_MDN_CsCh_Fly8_002_SG1_behData_images'
root_folder = '/data/LiftFly3D/DF3D/'

Plot DF3D prediction on images and save as a 25fps video

In [3]:
G, color_edge = skeleton()
poses = pickle.load(open(root_folder + 'pose_result__data_paper_' + experiment + '.pkl', "rb"))
poses = poses['points2d']
poses.shape

(7, 900, 38, 2)

In [13]:
cam=1 #5 #run twice for cam 1 and 5

#load images
files = glob.glob(root_folder + experiment + '/' + 'camera_' + str(cam) + '*.jpg')
imgs = []
for f in sorted(files):
    img = cv2.imread(f)
    if cam == 5:
        img = np.fliplr(img)
    imgs.append(img)

fig = plt.figure()
ax = fig.add_subplot(111)

metadata = dict(title='camera_' + str(cam), artist='LiftFly3D', comment='Watch this!')
writer = FFMpegWriter(fps=25, metadata=metadata)
with writer.saving(fig, root_folder + experiment + "/camera_" + str(cam) + ".mp4", 100):
    for t in range(poses.shape[1]):
        ax.cla()
        
        ax.imshow(imgs[t])
        
        for i, j in enumerate(G.edges()): 
            x = np.array((poses[cam,t,j[0],0], poses[cam,t,j[1],0]))
            y = np.array((poses[cam,t,j[0],1], poses[cam,t,j[1],1]))
                   
            ax.plot(x, y, c=color_edge[i], alpha=1.0, linewidth = 2)
            
        ax.set_axis_off()    
        
        writer.grab_frame()

In [ ]:
#flip camera5 horizontally 
#ffmpeg -i camera_5.mp4 -vf hflip -c:a copy camera_5.mp4

#stack videos vertically (call this in the folder where the images are)
#ffmpeg -i camera_1.mp4 -i camera_5.mp4 -filter_complex vstack=inputs=2 stacked.mp4

#stack videos horizontally (call this in the folder where the images are)
#ffmpeg -i prediction_cams.mp4 -i stacked.mp4 -filter_complex "[0:v]scale=-1:800[v0];[v0][1:v]hstack=inputs=2" output.mp4